## import

In [1]:
import argparse
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score
import cv2
import numpy as np
import os
from os import path
import pandas as pd
import csv

## Preprocessing for Video

#### trian_image

In [3]:
train_csv = pd.read_csv("../input/train_list.csv", header=None)
i = 0
count_row = 0
for  row in train_csv.iloc[:,0]:
    #print(path.exists('../input/train/%s' % row))
    cap = cv2.VideoCapture('../input/train/%s' % row)
    x=0
    while(cap.isOpened()):
        flag, frame = cap.read()
        if flag == False:
            break
        #has meteor
        if train_csv.iloc[count_row,1] == 1:
            cv2.imwrite("../input/hmeteor_train/"+"img_%s.png" % str(i).zfill(7), frame)
            i += 1
            with open('../input/im_train.csv', 'a') as f:
                writer = csv.writer(f, lineterminator='\n')
                writer.writerow(["%s" % str("img_%s.png" % str(i).zfill(7)),int(train_csv.iloc[count_row,1])])
        
        #hasnt meteor
        elif x % 60 == 0:
            cv2.imwrite("../input/image_train/"+"img_%s.png" % str(i).zfill(7), frame)
            #print('Save', "img_%s.png" % str(i).zfill(5))
            i += 1
            with open('../input/im_train.csv', 'a') as f:
                writer = csv.writer(f, lineterminator='\n')
                writer.writerow(["%s" % str("img_%s.png" % str(i).zfill(7)),int(train_csv.iloc[count_row,1])])
        x+=1
    cap.release()
    count_row += 1

























































































#### Meteor Data

In [32]:
train_csv = pd.read_csv("../input/train_list.csv", header=None)
i = 0
count_row = 0
for  row in train_csv.iloc[:,0]:
    if train_csv.iloc[count_row,1] == 1:
        cap = cv2.VideoCapture('../input/train/%s' % row)
        while(cap.isOpened()):
            flag, frame = cap.read()
            if flag == False:
                break
            cv2.imwrite("../input/meteor_train/"+"img_%s.png" % str(i).zfill(7), frame)
            print('Save', "img_%s.png" % str(i).zfill(5))
            with open('../input/meteor_train.csv', 'a') as f:
                writer = csv.writer(f, lineterminator='\n')
                writer.writerow(["%s" % str("img_%s.png" % str(i).zfill(7)),int(train_csv.iloc[count_row,1])])
            i += 1
        cap.release()
    count_row += 1

Save img_00000.png
Save img_00001.png
Save img_00002.png
Save img_00003.png
Save img_00004.png
Save img_00005.png
Save img_00006.png
Save img_00007.png
Save img_00008.png
Save img_00009.png
Save img_00010.png
Save img_00011.png
Save img_00012.png
Save img_00013.png
Save img_00014.png
Save img_00015.png
Save img_00016.png
Save img_00017.png
Save img_00018.png
Save img_00019.png
Save img_00020.png
Save img_00021.png
Save img_00022.png
Save img_00023.png
Save img_00024.png
Save img_00025.png
Save img_00026.png
Save img_00027.png
Save img_00028.png
Save img_00029.png
Save img_00030.png
Save img_00031.png
Save img_00032.png
Save img_00033.png
Save img_00034.png
Save img_00035.png
Save img_00036.png
Save img_00037.png
Save img_00038.png
Save img_00039.png
Save img_00040.png
Save img_00041.png
Save img_00042.png
Save img_00043.png
Save img_00044.png
Save img_00045.png
Save img_00046.png
Save img_00047.png
Save img_00048.png
Save img_00049.png
Save img_00050.png
Save img_00051.png
Save img_000

## 動的背景差分

手順
1. 記事を元にして差分アルゴリズムをくむ．
2. 背景を黒にする．
3. 動体抽出
3. 流星を抽出
https://qiita.com/hitomatagi/items/f9d4d6b816d729132231

In [ ]:
# 定数定義
ESC_KEY = 27     # Escキー
INTERVAL= 33     # インターバル
FRAME_RATE = 30  # fps

WINDOW_ORG = "org"
WINDOW_BACK = "back"
WINDOW_DIFF = "diff"

FILE_ORG = "org_768x576.avi"

# ウィンドウの準備
cv2.namedWindow(WINDOW_ORG)
cv2.namedWindow(WINDOW_BACK)
cv2.namedWindow(WINDOW_DIFF)

# 元ビデオファイル読み込み
mov_org = cv2.VideoCapture(FILE_ORG)

# 最初のフレーム読み込み
has_next, i_frame = mov_org.read()

# 背景フレーム
back_frame = np.zeros_like(i_frame, np.float32)

# 変換処理ループ
while has_next == True:
    # 入力画像を浮動小数点型に変換
    f_frame = i_frame.astype(np.float32)

    # 差分計算
    diff_frame = cv2.absdiff(f_frame, back_frame)

    # 背景の更新
    cv2.accumulateWeighted(f_frame, back_frame, 0.025)

    # フレーム表示
    cv2.imshow(WINDOW_ORG, i_frame)
    cv2.imshow(WINDOW_BACK, back_frame.astype(np.uint8))
    cv2.imshow(WINDOW_DIFF, diff_frame.astype(np.uint8))

    # Escキーで終了
    key = cv2.waitKey(INTERVAL)
    if key == ESC_KEY:
        break

    # 次のフレーム読み込み
    has_next, i_frame = mov_org.read()

# 終了処理
cv2.destroyAllWindows()
mov_org.release()

## modeling

In [1]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.optimizers import Adam

from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.utils import plot_model
from keras.callbacks import TensorBoard

from keras.utils import np_utils

Using TensorFlow backend.


In [ ]:
# モデルの定義
model = Sequential()

model.add(Conv2D(32,3,input_shape=(224,224,1)))
model.add(Activation('relu'))
model.add(Conv2D(16,3))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(3,2)))

model.add(Conv2D(64,3))
model.add(Activation('relu'))
model.add(Conv2D(32,3))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(3,2)))

model.add(Conv2D(128,3))
model.add(Activation('relu'))
model.add(Conv2D(128, 3))
model.add(Activation('relu'))
model.add(Conv2D(64,3))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(3,2)))

model.add(Conv2D(256,3))
model.add(Activation('relu'))
model.add(Conv2D(256,3))
model.add(Activation('relu'))
model.add(Conv2D(128, 3))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(3,2)))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(1.0))

model.add(Dense(nb_classes, activation='softmax'))

adam = Adam(lr=1e-4)

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch, verbose=1, validation_split=0.1)

## submit_data

#### test_image

In [31]:
i = 0
count_row = 0
for  file in os.listdir("../input/test"):
    #print(path.exists("../input/test/"+"01%s.mpg" % str(i).zfill(4)))
    i +=1
    cap = cv2.VideoCapture("../input/test/"+"01%s.mpg" % str(i).zfill(4))
    x=0
    while(cap.isOpened()):
        flag, frame = cap.read()
        if flag == False:
            break
        if x % 6 == 0:
            cv2.imwrite("../input/image_train/"+"img_%s.png" % str(i).zfill(7), frame)
            #print('Save', "img_%s.png" % str(i).zfill(5))
            with open('../input/im_train.csv', 'a') as f:
                writer = csv.writer(f, lineterminator='\n')
                writer.writerow(["%s" % str("img_%s.png" % str(i).zfill(7)),int(train_csv.iloc[count_row,1])])
            i += 1
        x+=1
    cap.release()
    count_row += 1

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [ ]:
with open('../input/submit.csv', 'a') as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow(["01%s.mpg" % str(i).zfill(4)])